In [1]:
import $cp.`/container/target/dependency/*`

import $cp.$

In [2]:
import org.apache.spark.sql.SparkSession

val spark = SparkSession.builder()
  .appName("ScalaSpark")
  .master("local[*]")
  .config("spark.sql.extensions", "io.delta.sql.DeltaSparkSessionExtension")
  .config("spark.sql.catalog.spark_catalog", "org.apache.spark.sql.delta.catalog.DeltaCatalog")
  .getOrCreate()

import spark.implicits._

val tablePath = "/tmp/delta-scala"


SLF4J: Failed to load class "org.slf4j.impl.StaticLoggerBinder".
SLF4J: Defaulting to no-operation (NOP) logger implementation
SLF4J: See http://www.slf4j.org/codes.html#StaticLoggerBinder for further details.


import org.apache.spark.sql.SparkSession
spark: SparkSession = org.apache.spark.sql.SparkSession@7c912823
import spark.implicits._
tablePath: String = "/tmp/delta-scala"

In [3]:
val df = Seq(
  (8, "bat"),
  (64, "mouse"),
  (1, "horse")
).toDF("number", "word")

df: org.apache.spark.sql.package.DataFrame = [number: int, word: string]

In [4]:
df.write
  .format("delta")
  .mode("overwrite")  // or append
  .save(tablePath)

SLF4J: Failed to load class "org.slf4j.impl.StaticMDCBinder".
SLF4J: Defaulting to no-operation MDCAdapter implementation.
SLF4J: See http://www.slf4j.org/codes.html#no_static_mdc_binder for further details.


In [5]:
spark.read.format("delta").load(tablePath).show()

+------+-----+
|number| word|
+------+-----+
|     1|horse|
|    64|mouse|
|     8|  bat|
+------+-----+

